Note: if you having trouble displaying the maps try running jupyter nbextension enable --py gmaps in your environment and retry.


Create a heat map that displays the humidity for every city from the part I of the homework.

Narrow down the DataFrame to find your ideal weather condition. For example:


A max temperature lower than 80 degrees but higher than 70.


Wind speed less than 10 mph.


Zero cloudiness.


Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.




Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.


Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.



In [34]:
# Dependencies
import requests
import json

# Google developer API key
from api_keys import g_key

In [42]:
#import dataframe weather1_data from 1st part of assignment (WeatherPy.ipynb)
%store -r weather1_data
weather1_data

,City Name,Latitude,Longitude,Temperature,Latitude Humidity,Cloudiness,Wind Speed
1,hithadhoo,-20.841587,85.603908,82.76,68.0,77.0,3.94
2,sao filipe,13.293488,-34.982080,75.88,60.0,0.0,6.08
4,bluff,-71.214122,172.363302,57.00,84.0,64.0,18.01
5,saint-philippe,-48.409808,62.306578,75.02,65.0,47.0,4.61
6,norman wells,68.737352,-120.152631,-13.00,77.0,90.0,3.51
...,...,...,...,...,...,...,...
495,ous,61.793925,61.297353,-17.07,88.0,100.0,6.85
496,qasigiannguit,66.282569,-48.212406,19.40,42.0,75.0,9.22
497,provideniya,62.015523,-174.457364,4.93,84.0,2.0,4.25
498,nanortalik,52.630018,-35.201380,22.64,89.0,0.0,11.48


In [ ]:
#Narrow down ideal conditions of city

#Ideal temperature Range
#A max temperature lower than 80 degrees but higher than 70.

#Ideal wind speed: 
#Wind speed less than 10 mph.

#Ideal cloudiness:
#Zero cloudiness.